In [4]:
%pip install patool pyunpack

     ---------------------------------------- 77.5/77.5 KB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\hp\OneDrive\Desktop\machine_learning\ml\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd 
from zipfile import ZipFile
import shutil 
import warnings
import numpy as np 
import patoolib 
import seaborn as sns 
from datetime import datetime 
import requests, os, json 

from simpledbf import Dbf5 
warnings.simplefilter('ignore')

In [2]:
try:
    os.mkdir('DATASETS')
except Exception as e:
    pass 

patoolib.extract_archive(r'C:\Users\hp\OneDrive\Desktop\lightning_project\LDS_Data_January to December 2022.rar', outdir = 'DATASETS')


patool: Extracting C:\Users\hp\OneDrive\Desktop\lightning_project\LDS_Data_January to December 2022.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "C:\Users\hp\OneDrive\Desktop\lightning_project\LDS_Data_January to December 2022.rar"
patool:     with cwd=DATASETS
patool: ... C:\Users\hp\OneDrive\Desktop\lightning_project\LDS_Data_January to December 2022.rar extracted to `DATASETS'.


'DATASETS'

In [17]:
def compiledbf(path = r'C:\Users\hp\OneDrive\Desktop\lightning_project\DATASETS\LDS_Data_January to December 2022'):
    directory = os.listdir(path)
    dfs = []
    for month_file in directory:
        month_path = os.path.join(path, month_file)
        days = os.listdir(month_path)

        for day_file in days:
            
            if day_file.endswith('.dbf') and not month_file.startswith('December'):

                day_path = os.path.join(month_path, day_file)
                date_info = day_file.split('_')[2]
                day, month, year = int(date_info[:2]), int(date_info[2:4]), int(date_info[4:]) 
                day_df = Dbf5(day_path).to_dataframe()

                day_df['DATE'] = datetime(year = year, month = month, day = day)
                dfs.append(day_df)

    return dfs 

dfs = compiledbf()
main_df = pd.concat(dfs)



In [29]:

main_df.to_csv('DATAFRAME/lightning.csv', index=False)
main_df['month'] = main_df.DATE.dt.month_name()

In [38]:
max_strikes = pd.DataFrame(main_df.groupby('month')['Count'].max()) 
max_strikes.reset_index(level = 0, inplace=True)

def getMaxDate(record):
    return main_df.loc[(main_df.month == record.month) & (main_df.Count == record.Count), 'DATE'].values[0]

max_strikes['DATE'] = max_strikes.apply(lambda x: getMaxDate(x), axis = 1)

In [40]:
max_strikes.to_csv('DATAFRAME/max_strikes.csv', index=False)

In [46]:
try:
    os.mkdir('DECEMBER_ZIP')
except Exception as E:
    pass 

def extractZips(path = r'C:\Users\hp\OneDrive\Desktop\lightning_project\December 2022 data'):
    
    for day_folder in os.listdir(path):

        if not day_folder.endswith('.rar'):
            folders = os.listdir(os.path.join(path, day_folder))
            if len(folders) >= 1:
                zip_path = os.path.join(os.path.join(path, day_folder), folders[0]) 
                shutil.copy(src = zip_path, dst = 'DECEMBER_ZIP')

    return 

extractZips()



In [51]:
try:
    os.mkdir('DECEMBER')
except Exception as E:
    pass 

def getVisual(path = 'DECEMBER_ZIP'):

    for zip_path in os.listdir(path):
        file_name = zip_path.split('.')[0] + '.dbf'
        f = ZipFile(f'DECEMBER_ZIP/{zip_path}', 'r')
        f.extractall('DECEMBER')
        f.close()

    return 

getVisual()

In [54]:
def getExcel(path = r'C:\Users\hp\OneDrive\Desktop\lightning_project\DECEMBER\shape'):
    dfs = []
    for dbf_file in os.listdir(path):

        if dbf_file.endswith('.dbf'):
            dbf_path = os.path.join(path, dbf_file)
            year, month, day = dbf_file.split('-')[1:4]
            df = Dbf5(dbf_path).to_dataframe()
            df['DATE'] = datetime(year = int(year), month = int(month), day = int(day))
            dfs.append(df)

    return dfs

dec = pd.concat(getExcel()).drop(columns=['id'])


In [56]:
dec.rename({'count':'Count'}, axis =1, inplace=True)

In [58]:
dec['month'] = dec.DATE.dt.month_name()

In [61]:
df = pd.concat([main_df, dec])
df.to_csv('DATASETS/lightning_2022.csv', index=False)

In [62]:
df

,cx,cy,gid,Count,DATE,month
0,72.75,20.15,G042347,0.0,2022-01-01,January
1,72.85,20.15,G042348,0.0,2022-01-01,January
2,72.95,20.15,G042349,0.0,2022-01-01,January
3,73.05,20.15,G042350,0.0,2022-01-01,January
4,73.15,20.15,G042351,0.0,2022-01-01,January
...,...,...,...,...,...,...
1561,72.05,24.55,G055540,0.0,2022-12-25,December
1562,73.25,20.35,G042952,0.0,2022-12-25,December
1563,72.15,24.55,G055541,0.0,2022-12-25,December
1564,73.05,20.45,G043250,0.0,2022-12-25,December
